# Enhancing Singapore Airlines' Service Through Automated Sentiment Analysis of Customer Reviews



**Motivation**



## Singapore Airlines Customer Reviews Dataset Information

The [Singapore Airlines Customer Reviews Dataset](https://www.kaggle.com/datasets/kanchana1990/singapore-airlines-reviews) aggregates 10,000 anonymized customer reviews, providing a broad perspective on the passenger experience with Singapore Airlines. 

The dimensions are shown below:
- **`published_date`**: Date and time of review publication.
- **`published_platform`**: Platform where the review was posted.
- **`rating`**: Customer satisfaction rating, from 1 (lowest) to 5 (highest).
- **`type`**: Specifies the content as a review.
- **`text`**: Detailed customer feedback.
- **`title`**: Summary of the review.
- **`helpful_votes`**: Number of users finding the review helpful.

## Additional web scraping of online reviews

During our EDA, we noticed two main trends in the distribution of our dataset:
1. Less than 10% of our reviews were published from the years 2022 to 2024, making it hard for us to capture recent trends in sentiment.
2. Most of the reviews were highly positive, which could mean that SIA had mostly positive reviews, nevertheless we wanted to get more information on negative reviews to improve the robustness of our model.

### TripAdvisor

We scraped more data for airline reviews from TripAdvisor, specifically for the years 2022 to 2024. 
(https://www.tripadvisor.com.sg/Airline_Review-d8729151-Reviews-Singapore-Airlines)

The dimensions are shown below:
- **`Year`**: Year of review publication.
- **`Month`**: Month of review publication.
- **`Title`**: Title of review publication.
- **`Review Text`**: Main text content of review publication.
- **`Rating`**: Numerical rating provided by reviewer (Scale: 1 to 5)


### Skytrax

We also scraped from Skytrax, which is another data source for online reviews. 
(https://www.airlinequality.com/airline-reviews/singapore-airlines/?sortby=post_date%3ADesc&pagesize=100)

The dimensions are shown below:
- **`Year`**: Year of review publication.
- **`Month`**: Month of review publication.
- **`Title`**: Title of review publication.
- **`Review Text`**: Main text content of review publication.
- **`Rating`**: Numerical rating provided by reviewer (Scale: 1 to 10)

## Importing Libraries

Please uncomment the code box below to pip install relevant dependencies for this notebook.

In [7]:
# !pip3 install -r requirements.txt

In [1]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np
from datetime import datetime 

# Statistical functions
from scipy.stats import zscore

# Text Preprocessing and NLP
import nltk
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords

# Tokenizing sentences/words
from nltk.corpus import wordnet

# Tokenizing sentences/words
from nltk.tokenize import word_tokenize
# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer


# For generating n-grams
from nltk.util import ngrams
from collections import Counter

## Data Preparation (Loading CSV)

Load the three CSV files into a pandas DataFrame `data`.

In [2]:
data = pd.read_csv('final_df.csv')

In [3]:
data.head()

,year,month,sentiment,processed_full_review
0,2024,3,Neutral,ok use airlin go singapor london heathrow issu...
1,2024,3,Negative,don give money book paid receiv email confirm ...
2,2024,3,Positive,best airlin world best airlin world seat food ...
3,2024,3,Negative,premium economi seat singapor airlin not worth...
4,2024,3,Negative,imposs get promis refund book flight full mont...


In [4]:
data['sentiment'].value_counts()

sentiment
Positive    7913
Negative    2441
Neutral     1164
Name: count, dtype: int64

In [5]:
data['year'].value_counts()

year
2019    5129
2018    2596
2022    1184
2023    1111
2020     888
2024     514
2021      96
Name: count, dtype: int64

# Basic RNN

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Parameters
vocab_size = 5000         # Limit vocabulary to 5000 words
embedding_dim = 128        # Embedding dimensions for each word
max_sequence_length = 300 # Max number of words in each sequence

# Step 1: Tokenize and Pad the Text
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(data['processed_full_review'])
sequences = tokenizer.texts_to_sequences(data['processed_full_review'])
X_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# Labels
sentiment_dict = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y = data['sentiment'].map(sentiment_dict).values

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Step 2: Define a Simple RNN Model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length, trainable=True))
model.add(SimpleRNN(64, activation='tanh'))
model.add(Dropout(0.5))  # Add dropout for regularization
model.add(Dense(3, activation='softmax'))   # Output layer for 3 classes

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 3: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=64,  validation_split=0.2, verbose=1)

y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate and print classification report
report = classification_report(y_test, y_pred, target_names=['Negative', 'Neutral', 'Positive'], zero_division=0, digits=4)
print('Performance Metrics:\n', report)

Epoch 1/10


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.6953 - loss: 0.7855 - val_accuracy: 0.8247 - val_loss: 0.4852
Epoch 2/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.8643 - loss: 0.3740 - val_accuracy: 0.8285 - val_loss: 0.4735
Epoch 3/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.9305 - loss: 0.2286 - val_accuracy: 0.8193 - val_loss: 0.5020
Epoch 4/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.9719 - loss: 0.1102 - val_accuracy: 0.8209 - val_loss: 0.6072
Epoch 5/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.9843 - loss: 0.0586 - val_accuracy: 0.8166 - val_loss: 0.6437
Epoch 6/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.9974 - loss: 0.0218 - val_accuracy: 0.8068 - val_loss: 0.6902
Epoch 7/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.9938 - loss: 0.0267 - val_accuracy: 0.8199 - val_loss: 0.7366
Epoch 8/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9993 - loss: 0.0114 - val_accuracy: 0.820

# RNN while accounting for imbalanced classes

Overall F1 score drops very slightly

By applying `class_weight` using `compute_class_weight`, the model pays more attention to minority classes, which may cause it to misclassify some instances of the majority class. This re-balncing can lower the overall F1 score if the model sacrifices performance on majority classes.

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

# Parameters
vocab_size = 5000         # Limit vocabulary to 5000 words
embedding_dim = 128        # Embedding dimensions for each word
max_sequence_length = 300 # Max number of words in each sequence

# Step 1: Tokenize and Pad the Text
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(data['processed_full_review'])
sequences = tokenizer.texts_to_sequences(data['processed_full_review'])
X_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# Labels
sentiment_dict = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y = data['sentiment'].map(sentiment_dict).values

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Step 2: Define a Simple RNN Model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length, trainable=True))
model.add(SimpleRNN(64, activation='tanh'))
model.add(Dropout(0.5))  # Add dropout for regularization
model.add(Dense(3, activation='softmax'))   # Output layer for 3 classes

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Step 3: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=64,  validation_split=0.2, verbose=1, class_weight=class_weights_dict)

y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate and print classification report
report = classification_report(y_test, y_pred, target_names=['Negative', 'Neutral', 'Positive'], zero_division=0, digits=4)
print('Performance Metrics:\n', report)

Epoch 1/10


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.4801 - loss: 1.0711 - val_accuracy: 0.7618 - val_loss: 0.6118
Epoch 2/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.8107 - loss: 0.6456 - val_accuracy: 0.7661 - val_loss: 0.5891
Epoch 3/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9174 - loss: 0.2804 - val_accuracy: 0.7944 - val_loss: 0.5659
Epoch 4/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9677 - loss: 0.1091 - val_accuracy: 0.7971 - val_loss: 0.6449
Epoch 5/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9725 - loss: 0.0751 - val_accuracy: 0.7878 - val_loss: 0.6871
Epoch 6/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9925 - loss: 0.0302 - val_accuracy: 0.7971 - val_loss: 0.7275
Epoch 7/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.9937 - loss: 0.0218 - val_accuracy: 0.7623 - val_loss: 0.8378
Epoch 8/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.9931 - loss: 0.0233 - val_accuracy: 0.794

# RNN + Count Vectoriser

### Loss of Sequential Information
Poor performance because RNNs are not well-suited to the bag-of-words representation generated by `CountVectorizer`. Since `CountVectorizer` treats each document as a set of words without any order, words are represented only by their counts, not by their position in the text. Since RNNs are designed to work with ordered sequences, where the position and context of words matter, without preserving word order, the RNN cannot capture dependencies between words over time.

### Sparse, non-contextual input
`CountVectorizer` produces a sparse representation where each word is treated as an independent feature based on its frequency. There is no semantic or contextual relationship between words, and the word counts lack dense, meaningful relationships that an RNN could leverage, since RNNs perform best with dense, continuous data that represents meaningful relationships between words, typically achieved with word embeddings.

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

# Parameters
max_features = 5000       # Limit vocabulary to 5000 words
max_sequence_length = 300 # Max number of words in each sequence

# Step 1: Text Vectorization using CountVectorizer
vectorizer = CountVectorizer(max_features=max_features)
X_counts = vectorizer.fit_transform(data['processed_full_review']).toarray()

# Convert Counts to Sequences
X_padded = pad_sequences(X_counts, maxlen=max_sequence_length)

# Labels
sentiment_dict = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y = data['sentiment'].map(sentiment_dict).values

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Reshape input to 3D for RNN (samples, timesteps, features)
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

model = Sequential()
model.add(SimpleRNN(64, activation='tanh', input_shape=(X_train_reshaped.shape[1], 1)))  # Input shape adjusted
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(3, activation='softmax'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

model.fit(X_train_reshaped, y_train, epochs=10, batch_size=64,  validation_split=0.2, verbose=1, class_weight=class_weights_dict)

y_pred_prob = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate and print classification report
report = classification_report(y_test, y_pred, target_names=['Negative', 'Neutral', 'Positive'], zero_division=0, digits=4)
print('Performance Metrics:\n', report)

Epoch 1/10


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.3603 - loss: 1.0976 - val_accuracy: 0.2409 - val_loss: 1.0996
Epoch 2/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.3883 - loss: 1.1024 - val_accuracy: 0.5144 - val_loss: 1.0559
Epoch 3/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.4197 - loss: 1.0939 - val_accuracy: 0.4330 - val_loss: 1.0663
Epoch 4/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.4909 - loss: 1.0910 - val_accuracy: 0.5279 - val_loss: 1.0767
Epoch 5/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.4377 - loss: 1.0823 - val_accuracy: 0.5301 - val_loss: 1.0364
Epoch 6/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.4464 - loss: 1.0977 - val_accuracy: 0.5383 - val_loss: 1.0831
Epoch 7/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.4647 - loss: 1.0799 - val_accuracy: 0.5513 - val_loss: 1.0820
Epoch 8/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.4520 - loss: 1.1047 - val_accuracy: 0.613

# RNN + Count Vectoriser + Conversion to pseudo-sequences with word indices

Performance is better than Basic RNN.

Over here, we transform the `CountVectorizer` output into integer sequences which is compatible with the embedding layer. 

Why `CountVectorizer` is better here is because sentiment analysis often hinges more on the presence of certain key words rather than on the strict order of words in a sequence. Unlike other NLP tasks where the exact sequence of words matters (e.g. translation or grammar correction), sentiment analysis can often succeed with just the occurrence or frequency of these key items. `CountVectorizer` captures this by creating a bag-of-words representation that prioritises word presence and frequency, which is often enough for sentiment detection.



In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

# Parameters
max_features = 5000       # Limit vocabulary to 5000 words
embedding_dim = 128        # Embedding dimensions for each word
max_sequence_length = 300 # Max number of words in each sequence

# Step 1: Text Vectorization using CountVectorizer
vectorizer = CountVectorizer(max_features=max_features)
X_counts = vectorizer.fit_transform(data['processed_full_review'])
word_index = vectorizer.vocabulary_

# Inverse vocabulary mapping for sequences creation
index_to_word = {i: word for word, i in word_index.items()}

def counts_to_sequences(X_counts):
    sequences = []
    for i in range(X_counts.shape[0]):
        indices = X_counts[i].nonzero()[1]
        words = [index_to_word[idx] for idx in indices]
        seq = [word_index[word] + 1 for word in words]  # +1 because 0 is reserved for padding
        sequences.append(seq)
    return sequences

sequences = counts_to_sequences(X_counts)
X_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# Labels
sentiment_dict = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y = data['sentiment'].map(sentiment_dict).values

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length, trainable=True))
model.add(SimpleRNN(64, activation='tanh', input_shape=(X_train_reshaped.shape[1], 1)))  # Input shape adjusted
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(3, activation='softmax'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

model.fit(X_train, y_train, epochs=10, batch_size=64,  validation_split=0.2, verbose=1, class_weight=class_weights_dict)

y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate and print classification report
report = classification_report(y_test, y_pred, target_names=['Negative', 'Neutral', 'Positive'], zero_division=0, digits=4)
print('Performance Metrics:\n', report)

Epoch 1/10


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 46ms/step - accuracy: 0.4413 - loss: 1.0788 - val_accuracy: 0.6234 - val_loss: 0.9673
Epoch 2/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.5211 - loss: 0.9851 - val_accuracy: 0.7835 - val_loss: 0.5874
Epoch 3/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.8911 - loss: 0.4458 - val_accuracy: 0.7900 - val_loss: 0.5219
Epoch 4/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9549 - loss: 0.2068 - val_accuracy: 0.8090 - val_loss: 0.5072
Epoch 5/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9822 - loss: 0.0892 - val_accuracy: 0.8166 - val_loss: 0.5471
Epoch 6/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9913 - loss: 0.0474 - val_accuracy: 0.8188 - val_loss: 0.5817
Epoch 7/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.9959 - loss: 0.0291 - val_accuracy: 0.8237 - val_loss: 0.6318
Epoch 8/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.9988 - loss: 0.0176 - val_accuracy: 0.816

# RNN + Within model trained Word2Vec

`Word2Vec` performs worse than `CountVectorizer`.

Because our dataset is only 10k rows, Word2Vec embeddings might lack the depth needed for nuanced sentiment patterns, particularly without pre-training on a larger corpus. If Word2Vec embeddings do not generalise well or have insufficient context, the RNN might not capture subtle sentiment signals in the text, which can degrade model performance. In contrast, CountVectorizer builds a fixed vocab of words based on frequency, and does not need to learn semantic relationships among words, making it robust in cases where the model vocab size is small. 


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from gensim.models import Word2Vec
import numpy as np

# Parameters
vocab_size = 5000         # Limit vocabulary to 5000 words
embedding_dim = 128        # Embedding dimensions for each word
max_sequence_length = 300 # Max number of words in each sequence

# Step 1: Tokenize and Pad the Text
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(data['processed_full_review'])
sequences = tokenizer.texts_to_sequences(data['processed_full_review'])
X_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# Labels
sentiment_dict = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y = data['sentiment'].map(sentiment_dict).values

sentences = [text.split() for text in data['processed_full_review']]
word2vec_model = Word2Vec(sentences, vector_size=embedding_dim, window=5, min_count=1, workers=4, sg=1)

# Create Embedding Matrix from Trained Word2Vec Model
embedding_matrix = np.zeros((vocab_size, embedding_dim))
word_index = tokenizer.word_index

for word, i in word_index.items():
    if i < vocab_size:
        # Retrieve the embedding vector for the word
        embedding_vector = word2vec_model.wv[word] if word in word2vec_model.wv else None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Step 2: Define a Simple RNN Model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=True))
model.add(SimpleRNN(64, activation='tanh'))
model.add(Dropout(0.5))  # Add dropout for regularization
model.add(Dense(3, activation='softmax'))   # Output layer for 3 classes

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Step 3: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=64,  validation_split=0.2, verbose=1, class_weight=class_weights_dict)

y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate and print classification report
report = classification_report(y_test, y_pred, target_names=['Negative', 'Neutral', 'Positive'], zero_division=0, digits=4)
print('Performance Metrics:\n', report)

c:\Users\Redbu\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 14s 90ms/step - accuracy: 0.5156 - loss: 1.0849 - val_accuracy: 0.7461 - val_loss: 0.6227
Epoch 2/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 10s 88ms/step - accuracy: 0.7304 - loss: 0.7668 - val_accuracy: 0.7428 - val_loss: 0.6520
Epoch 3/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 10s 84ms/step - accuracy: 0.7818 - loss: 0.6514 - val_accuracy: 0.7184 - val_loss: 0.6758
Epoch 4/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.8472 - loss: 0.4936 - val_accuracy: 0.6815 - val_loss: 0.8051
Epoch 5/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 10s 84ms/step - accuracy: 0.8888 - loss: 0.3527 - val_accuracy: 0.7016 - val_loss: 0.7845
Epoch 6/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 11s 91ms/step - accuracy: 0.9197 - loss: 0.2260 - val_accuracy: 0.8128 - val_loss: 0.6503
Epoch 7/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 10s 82ms/step - accuracy: 0.8503 - loss: 0.3032 - val_accuracy: 0.7868 - val_loss: 0.6955
Epoch 8/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 10s 84ms/step - accuracy: 0.9620 - loss: 0.1100 - 

# RNN + Pre-trained Word2Vec


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from gensim.models import KeyedVectors
import numpy as np

# Parameters
vocab_size = 5000         # Limit vocabulary to 5000 words
embedding_dim = 300        # Embedding dimensions for each word
max_sequence_length = 300 # Max number of words in each sequence

# Step 1: Tokenize and Pad the Text
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(data['processed_full_review'])
sequences = tokenizer.texts_to_sequences(data['processed_full_review'])
X_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# Labels
sentiment_dict = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y = data['sentiment'].map(sentiment_dict).values

word2vec_model = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

# Create Embedding Matrix with Pre-trained Word2Vec
embedding_matrix = np.zeros((vocab_size, embedding_dim))
word_index = tokenizer.word_index

for word, i in word_index.items():
    if i < vocab_size:
        # Retrieve the embedding vector for the word
        if word in word2vec_model:
            embedding_matrix[i] = word2vec_model[word]

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Step 2: Define a Simple RNN Model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(SimpleRNN(64, activation='tanh'))
model.add(Dropout(0.5))  # Add dropout for regularization
model.add(Dense(3, activation='softmax'))   # Output layer for 3 classes

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Step 3: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=64,  validation_split=0.2, verbose=1, class_weight=class_weights_dict)

y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate and print classification report
report = classification_report(y_test, y_pred, target_names=['Negative', 'Neutral', 'Positive'], zero_division=0, digits=4)
print('Performance Metrics:\n', report)

c:\Users\Redbu\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 13s 80ms/step - accuracy: 0.4800 - loss: 1.0975 - val_accuracy: 0.7401 - val_loss: 0.6825
Epoch 2/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 9s 76ms/step - accuracy: 0.6848 - loss: 0.8667 - val_accuracy: 0.6923 - val_loss: 0.7555
Epoch 3/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - accuracy: 0.6901 - loss: 0.8259 - val_accuracy: 0.7352 - val_loss: 0.6554
Epoch 4/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 8s 70ms/step - accuracy: 0.6912 - loss: 0.8376 - val_accuracy: 0.7016 - val_loss: 0.7509
Epoch 5/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 8s 70ms/step - accuracy: 0.7443 - loss: 0.7090 - val_accuracy: 0.7314 - val_loss: 0.6768
Epoch 6/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.7573 - loss: 0.6879 - val_accuracy: 0.7488 - val_loss: 0.6388
Epoch 7/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 8s 70ms/step - accuracy: 0.7775 - loss: 0.6320 - val_accuracy: 0.6983 - val_loss: 0.7235
Epoch 8/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 8s 70ms/step - accuracy: 0.8022 - loss: 0.5752 - val_acc